In [3]:
import pandas as pd
import geopandas as gpd
import folium
import os
import glob
import branca.colormap as cm
import random
from folium.features import GeoJsonTooltip, GeoJsonPopup

#### 1. 인구 데이터 불러오기

In [4]:
pop_df = pd.read_csv("C:/Users/hk522/Desktop/캡스톤 팀프로젝트/행정안전부_지역별(법정동) 성별 연령별 주민등록 인구수_20250331.csv", encoding='euc-kr')

### 2. 2글자 시도명 → 법정동 시도명 매핑

In [5]:
sido_mapping = {
    '서울': '서울특별시',
    '부산': '부산광역시',
    '대구': '대구광역시',
    '인천': '인천광역시',
    '광주': '광주광역시',
    '대전': '대전광역시',
    '울산': '울산광역시',
    '세종': '세종특별시',
    '경기': '경기도',
    '강원': '강원도',
    '충북': '충청북도',
    '충남': '충청남도',
    '전북': '전라북도',
    '전남': '전라남도',
    '경북': '경상북도',
    '경남': '경상남도',
    '제주': '제주도'
}

### 3. 법정동 공간정보 폴더 경로

In [6]:
root_folder = r"C:\Users\hk522\Desktop\캡스톤 팀프로젝트\법정동 공간정보"

### 4. 저장할 폴더 만들기

In [7]:

os.makedirs('C:/Users/hk522/Desktop/캡스톤 팀프로젝트/시도별지도', exist_ok=True)

#### 5. 시도별 폴더 목록 가져오기

In [8]:
sido_folders = [f.path for f in os.scandir(root_folder) if f.is_dir()]


### 6. 시도별 shp 파일 처리

In [9]:
# -----------------------------
# 1. 시군구 경계 스타일 (검정색)
# -----------------------------
def sigungu_border_style(feature):
    return {
        'fillColor': 'none',
        'color': 'black',  # 시군구 경계 검정색
        'weight': 1,
        'fillOpacity': 0
    }

# -----------------------------
# 2. 시도 경계 스타일 (파란색)
# -----------------------------
def sido_border_style(feature):
    return {
        'fillColor': 'none',
        'color': 'blue',  # 시도 경계 파란색
        'weight': 2,
        'fillOpacity': 0
    }

In [10]:
# -----------------------------
# 3. 색상 팔레트 (랜덤 색상 선택)
# -----------------------------
colormap_list = [
    cm.linear.Blues_09,   # 파랑 계열
    cm.linear.Greens_09,  # 초록 계열
    cm.linear.Purples_09, # 보라 계열
    cm.linear.Reds_09,    # 빨강 계열
    cm.linear.Oranges_09  # 주황 계열
]

In [19]:
# -----------------------------
# 4. 지도 생성 함수
# -----------------------------
def create_map(geo_df, output_path):
    # 지도 초기화
    center = geo_df.geometry.centroid.unary_union.centroid
    m = folium.Map(location=[center.y, center.x], zoom_start=7)

    # 4-1. 시군구별 랜덤 색상 적용
    for sigungu in geo_df['시군구명'].unique():
        subset = geo_df[geo_df['시군구명'] == sigungu]
        subset = subset[subset['계'] > 0]        # 인구 0제외
        if subset.empty:
            continue

        # 랜덤 컬러맵 + 해당 시군구 min/max 스케일
        cmap = random.choice(colormap_list).scale(
            subset['계'].min(), subset['계'].max()
        )

        def style_fn(feat, cmap=cmap):
            val = feat['properties']['계']
            return {
                'fillColor': cmap(val),
                'color': 'black',  # 시군구 경계선
                'weight': 1,
                'fillOpacity': 0.7
            }

        folium.GeoJson(
            subset.to_json(),
            style_function=style_fn,
            tooltip=GeoJsonTooltip(
                fields=['EMD_NM','계','시군구명'],
                aliases=['읍면동명','인구수','시군구'],
                localize=True, sticky=True, labels=True, toLocaleString=True
            ),
            popup=GeoJsonPopup(
                fields=['EMD_NM','계','시군구명'],
                aliases=['읍면동명','인구수','시군구'],
                localize=True, labels=True, toLocaleString=True
            )
        ).add_to(m)

    # 4-2. 시도별 경계 추가
    sido_merged = geo_df.dissolve(by='시도명').reset_index()
    folium.GeoJson(
        sido_merged.to_json(),
        style_function=sido_border_style,
        tooltip=GeoJsonTooltip(
            fields=['시도명'],
            aliases=['시도'],
            localize=True, sticky=True, labels=True
        )
    ).add_to(m)

    # 4-3. 저장
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    m.save(output_path)

In [ ]:
# -----------------------------
# 5. 시도별 지도 생성 및 저장
# -----------------------------
merged_list = []

for sido_folder in sido_folders:
    folder_name = os.path.basename(sido_folder)
    folder_short = folder_name[-2:]  # 폴더명 끝 2글자
    sido_name = sido_mapping.get(folder_short)

    shp_files = glob.glob(os.path.join(sido_folder, '*.shp'))
    shp_file = shp_files[0]
    geo_df = gpd.read_file(shp_file, encoding='euc-kr')

    # 인구 데이터 해당 시도만
    pop_sido_df = pop_df[pop_df['시도명'] == sido_name].copy()
    pop_sido_df['읍면동코드'] = pop_sido_df['법정동코드'].astype(str).str[:8]
    pop_grouped = pop_sido_df.groupby(['시군구명', '읍면동코드'])['계'].sum().reset_index()

    # 시도명 컬럼 추가
    pop_grouped['시도명'] = sido_name  # 시도명 추가

    # 공간정보와 인구수 병합
    merged = geo_df.merge(pop_grouped, left_on='EMD_CD', right_on='읍면동코드')

    # 좌표계 변환 (EPSG:4326)
    merged = merged.to_crs(epsg=4326)

    merged_list.append(merged)

    # 시도별 지도 저장
    create_map(merged, 'C:/Users/hk522/Desktop/캡스톤 팀프로젝트/시도별지도/{sido_name}_인구_지도.html')


# -----------------------------
# 6. 전국 지도 생성 및 저장
# -----------------------------
all_merged = gpd.GeoDataFrame(pd.concat(merged_list, ignore_index=True))
all_merged = all_merged.set_crs(merged_list[0].crs)  # 좌표계 설정

# 전체 지도 저장
create_map(all_merged, 'C:/Users/hk522/Desktop/캡스톤 팀프로젝트/시도별지도/전국_인구_지도.html')